In [12]:
! pip install pandas numpy pyodbc python-dotenv openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ---------------------------------------- 0.0/454.8 kB ? eta -:--:--
   --------------------------------------- 454.8/454.8 kB 14.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/96.0 kB ? eta -:--:--
   ---------------------------------------- 96.0/96.0 kB ? eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   ---------------------------------------- 0.0/73.5 kB ? eta -:--:--
   ---------------------------------------- 73.5/73.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/78.6 kB ? eta -:--:--
   ---------------------------------------- 78.6/78.6 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.7 kB ? eta -:--:--
   -----------------------------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd 
import pyodbc

def get_connection(driver, server, database):
    """Create a connection to the SQL Server database."""
    try:
        connection_string = f"""
            DRIVER={driver};
            SERVER={server};
            DATABASE={database};
            Trusted_Connection=yes;
            """
        connection = pyodbc.connect(connection_string)
        return connection
    except Exception as e:
        print(f"Error: {e}")
    
driver = 'SQL Server'
database = 'AdventureWorks2022'
server = 'ATISL400'



In [6]:
conn = get_connection(driver, server, database)

In [8]:
q = f"""
SELECT 
    TABLE_SCHEMA,
    TABLE_NAME
FROM 
    INFORMATION_SCHEMA.TABLES
WHERE 
    TABLE_TYPE = 'BASE TABLE'
    AND TABLE_SCHEMA = 'Purchasing'  -- replace 'dbo' with your schema name
ORDER BY 
    TABLE_NAME;
    """
    
with conn.cursor() as cursor:
    cursor.execute(q)
    schema = cursor.fetchall()

schema
# df = pd.DataFrame(schema, columns=[column[0] for column in cursor.description])

[('Purchasing', 'ProductVendor'),
 ('Purchasing', 'PurchaseOrderDetail'),
 ('Purchasing', 'PurchaseOrderHeader'),
 ('Purchasing', 'ShipMethod'),
 ('Purchasing', 'Vendor')]

In [ ]:
import pyodbc
import sqlite3
 
# Connect to SQL Server and restore the .bak file
# sql_server_conn_str = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=YourServerName;DATABASE=master;UID=YourUsername;PWD=YourPassword'
# with pyodbc.connect(sql_server_conn_str) as sql_conn:
#     sql_cursor = sql_conn.cursor()
#     sql_cursor.execute("RESTORE DATABASE YourDatabaseName FROM DISK = 'C:\\YourBackupFile.bak' WITH REPLACE")
conn = get_connection(driver, server, database)

with conn as sql_conn:
    sql_cursor = sql_conn.cursor()
    sql_cursor.execute("RESTORE DATABASE AdventureWorks2022 FROM DISK = 'C:\\Users\\Chirantan_Degloorkar\\Downloads\\AdventureWorks2022.bak' WITH REPLACE")
# Connect to the restored database and extract data
with conn as sql_conn:
    sql_cursor = sql_conn.cursor()
    sql_cursor.execute("SELECT * FROM YourTableName")
    rows = sql_cursor.fetchall()
 
# Create a new SQLite database
sqlite_conn = sqlite3.connect('your_sqlite_db.db')
sqlite_cursor = sqlite_conn.cursor()
sqlite_cursor.execute("CREATE TABLE YourTableName (col1, col2, ...)")
 
# Insert data into the SQLite database
sqlite_cursor.executemany("INSERT INTO YourTableName VALUES (?, ?, ...)", rows)
sqlite_conn.commit()
sqlite_conn.close()

In [9]:
! pip install tqdm

     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.7 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.7 kB 217.9 kB/s eta 0:00:01
     -------------------------- ----------- 41.0/57.7 kB 326.8 kB/s eta 0:00:01
     -------------------------------------- 57.7/57.7 kB 302.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   -------------------- ------------------- 41.0/78.5 kB 1.9 MB/s eta 0:00:01
   ------------------------------------ --- 71.7/78.5 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 78.5/78.5 kB 875.6 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pyodbc
import sqlite3
import pandas as pd
from tqdm import tqdm

def get_all_tables(conn):
    query = """
    SELECT 
        TABLE_SCHEMA,
        TABLE_NAME 
    FROM 
        INFORMATION_SCHEMA.TABLES 
    WHERE 
        TABLE_TYPE = 'BASE TABLE'
    """
    return pd.read_sql(query, conn)

def create_sqlite_table(df, table_name, sqlite_conn):
    # Create table in SQLite using pandas
    df.to_sql(table_name, sqlite_conn, if_exists='replace', index=False)

def migrate_database():
    # SQL Server connection
    sql_server_conn = pyodbc.connect(
        'DRIVER={SQL Server};'
        'SERVER=ATISL400;'  # Replace with your server name
        'DATABASE=AdventureWorks2022;'
        'Trusted_Connection=yes;'
    )

    # SQLite connection
    sqlite_conn = sqlite3.connect('adventureworks.db')

    try:
        # Get all tables
        tables_df = get_all_tables(sql_server_conn)
        
        print(f"Found {len(tables_df)} tables to migrate")
        
        # Iterate through each table
        for _, row in tqdm(tables_df.iterrows(), total=len(tables_df)):
            schema = row['TABLE_SCHEMA']
            table = row['TABLE_NAME']
            full_table_name = f"{schema}.{table}"
            
            try:
                print(f"\nMigrating table: {full_table_name}")
                
                # Read data from SQL Server
                query = f"SELECT * FROM {full_table_name}"
                df = pd.read_sql(query, sql_server_conn)
                
                # Create table in SQLite
                sqlite_table_name = f"{schema}_{table}"
                create_sqlite_table(df, sqlite_table_name, sqlite_conn)
                
                print(f"Successfully migrated {len(df)} rows from {full_table_name}")
                
            except Exception as e:
                print(f"Error migrating table {full_table_name}: {str(e)}")
                continue

    except Exception as e:
        print(f"Migration failed: {str(e)}")
        
    finally:
        sql_server_conn.close()
        sqlite_conn.close()

if __name__ == "__main__":
    migrate_database()

C:\Users\Chirantan_Degloorkar\AppData\Local\Temp\ipykernel_20844\2069422936.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


Found 71 tables to migrate


  0%|          | 0/71 [00:00<?, ?it/s]C:\Users\Chirantan_Degloorkar\AppData\Local\Temp\ipykernel_20844\2069422936.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, sql_server_conn)



Migrating table: Sales.SalesTaxRate
Successfully migrated 29 rows from Sales.SalesTaxRate


  1%|▏         | 1/71 [00:00<00:07,  9.81it/s]

...

Successfully migrated 27647 rows from Sales.SalesOrderHeaderSalesReason

Migrating table: Sales.SalesPerson
Successfully migrated 17 rows from Sales.SalesPerson

Migrating table: Production.Illustration
Successfully migrated 5 rows from Production.Illustration

Migrating table: HumanResources.JobCandidate
Successfully migrated 13 rows from HumanResources.JobCandidate

Migrating table: Production.Location
Successfully migrated 14 rows from Production.Location

Migrating table: Person.Password


 97%|█████████▋| 69/71 [00:29<00:00,  2.45it/s]

Successfully migrated 19972 rows from Person.Password

Migrating table: Sales.SalesPersonQuotaHistory
Successfully migrated 163 rows from Sales.SalesPersonQuotaHistory

Migrating table: Person.Person


100%|██████████| 71/71 [00:30<00:00,  2.34it/s]

Successfully migrated 19972 rows from Person.Person

Migrating table: Sales.SalesReason
Successfully migrated 10 rows from Sales.SalesReason
